<a href="https://colab.research.google.com/github/ajakhmol/SoftwareEvaluation/blob/main/EvaluationRecommendationEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# prompt: write a iterator on the criteria_list list and pass the value of the list in the generate_response function
import pandas as pd
from IPython.display import display, HTML
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from google.colab import drive
# Set the display width to control the output width
# pd.set_option('display.width', 500)

from tenacity import retry, wait_random_exponential, stop_after_attempt

In [8]:
criteria_list ={
    "Scalability and Performance": [
        "Concurrent user capacity (10,000+, 10% annual growth)",
        "Data storage and growth management (15% annual increase)",
        "API throughput and optimization",
        "Caching strategies and CDN",
        "Load balancing and failover",
        "Performance monitoring and alerting",
        "Database optimization and indexing",
        "Horizontal vs. vertical scaling strategies",
        "Capacity planning for peak usage"
    ],
    "Security and Compliance": [
        "Data encryption (at rest and in transit)",
        "Access control (roles, sharing, permissions)",
        "Multi-factor authentication (MFA)",
        "Compliance (GDPR, CCPA, GLBA, HIPAA, SOX)",
        "Vulnerability management and penetration testing",
        "Security auditing and logging",
        "Data masking and anonymization",
        "Security certifications and attestations",
        "Incident response plan"
    ],
    "User Experience and Adoption": [
        "Intuitive navigation and UI",
        "Mobile responsiveness and accessibility (WCAG 2.1)",
        "Personalization (dynamic content, AI recommendations)",
        "User onboarding and training",
        "Feedback mechanisms and surveys",
        "Gamification and community features",
        "Multi-language support",
        "User experience testing and optimization",
        "Accessibility testing with assistive technologies"
    ],
    "Integration and Ecosystem": [
        "Integration with existing systems (ERP, CRM, core banking)",
        "API-led connectivity (REST/SOAP)",
        "Data synchronization and transformation",
        "AppExchange ecosystem",
        "Identity provider integration (SSO)",
        "Data mapping and migration",
        "Managing technical debt",
        "Integration architecture and design patterns",
        "API governance and security"
    ],
    "Administration and Governance": [
        "User lifecycle management",
        "Permission set and profile management",
        "Content moderation and version control",
        "Audit trail management",
        "Change management",
        "Data governance",
        "Environment management",
        "Metadata management",
        "Deployment automation"
    ],
    "Support and Training": [
        "Support channels (Salesforce, managed services)",
        "Training resources and documentation",
        "In-app guidance",
        "Community knowledge bases",
        "SLAs",
        "Knowledge transfer",
        "Ongoing training",
        "Training for administrators and developers",
        "Certification programs"
    ],
    "Disaster Recovery and Uptime": [
        "Disaster recovery plan",
        "Data backup and restoration",
        "Uptime SLAs",
        "Redundancy",
        "Business continuity",
        "DR drills",
        "Monitoring and alerting",
        "Recovery Time Objective (RTO) and Recovery Point Objective (RPO)",
        "Failover testing"
    ],
    "Customization and Flexibility": [
        "Lightning Web Components and Apex",
        "Declarative customization",
        "AppExchange integration",
        "Branding and theming",
        "API extensibility",
        "Managing custom code",
        "Scalability of customizations",
        "Customization governance",
        "Code review and testing processes"
    ],
    "Cost Optimization": [
        "Licensing model",
        "Resource utilization",
        "TCO analysis",
        "Value engineering",
        "Cloud infrastructure costs",
        "Maintenance costs",
        "ROI analysis",
        "Optimizing storage and compute resources",
        "Negotiating contracts with Salesforce"
    ],
    "AI, LLM, and LCM Integration": [
        "Salesforce Einstein",
        "LLM use cases (chatbots, content)",
        "LCM integration",
        "Data security and privacy for AI/ML",
        "Ethical implications of AI",
        "Future AI/ML roadmap",
        "AI-driven personalization",
        "Explainable AI and model interpretability",
        "AI model monitoring and retraining"
    ],
    "Regulatory and Political Landscape (US)": [
        "Data privacy regulations",
        "Financial industry regulations",
        "Political climate",
        "Cybersecurity regulations",
        "Data residency",
        "Legal and compliance reviews",
        "Monitoring regulatory changes",
        "Impact of international regulations",
        "Lobbying and advocacy efforts"
    ],
    "Salesforce Roadmap and Future Plans": [
        "Product roadmap analysis",
        "Future features",
        "Alignment with institution's strategy",
        "Upgrades and migrations",
        "Strategic partnerships",
        "Innovation assessment",
        "Emerging technologies",
        "Beta programs and early access features",
        "Influence of industry trends on Salesforce roadmap"
    ],
    "Resource Reskilling": [
        "Identifying skill gaps",
        "Training programs for Salesforce development, administration, and architecture",
        "Certification programs",
        "Mentorship programs",
        "Knowledge transfer from legacy system experts",
        "Change management strategies for user adoption",
        "Dedicated budget for training and development",
        "Cross-training existing resources",
        "Hiring new talent with Salesforce expertise"
    ],
    "Data Migration and Management": [
        "Data extraction from legacy systems (Java/Angular/Oracle)",
        "Data cleansing and transformation",
        "Data loading into Experience Cloud",
        "Data validation and reconciliation",
        "Data governance and quality",
        "Data archiving and retention policies",
        "Data security during migration",
        "Data mapping and schema design",
        "Performance testing of data migration",
        "Data migration tools and strategies"
    ],
    "Change Management and User Adoption": [
        "Communication plan for stakeholders",
        "User training and enablement programs",
        "Change impact assessment",
        "User acceptance testing (UAT)",
        "Feedback mechanisms and surveys",
        "Support for early adopters",
        "Post-implementation review and optimization",
        "Addressing user resistance to change",
        "Incentive programs for adoption",
        "User feedback analysis and action planning"
    ],
    "Vendor Management and Relationship": [
        "Salesforce contract negotiation",
        "Service Level Agreements (SLAs)",
        "Escalation procedures",
        "Communication channels",
        "Performance reviews",
        "Relationship management",
        "Dependency on Salesforce and vendor lock-in mitigation",
        "Data ownership and access rights",
        "Exit strategy",
        "Regular vendor meetings and communication"
    ],
    "Project Management and Implementation": [
        "Project scope definition",
        "Timeline and milestones",
        "Resource allocation",
        "Risk management",
        "Budget management",
        "Project governance",
        "Agile or Waterfall methodologies",
        "Regular project status reporting",
        "Quality assurance processes",
        "Project tracking and issue resolution"
    ],
    "Innovation and Future-Proofing": [
        "Staying current with Salesforce releases and updates",
        "Exploring emerging technologies (e.g., blockchain, Web3)",
        "Evaluating potential integrations with other platforms",
        "Developing a roadmap for future enhancements",
        "Participating in Salesforce community and events",
        "Benchmarking against industry best practices",
        "Building a culture of innovation within the team",
        "Experimenting with new features in sandbox environments",
        "Monitoring industry trends and adapting the platform accordingly",
        "Continuous improvement and optimization of the platform"
    ]
}



In [9]:
# Importing the necessary library for mounting Google Drive

# Mounting Google Drive to the Colab environment
drive.mount('/content/drive', force_remount=True)
# Read the API key from the text file and strip any leading or trailing whitespace
with open("/content/drive/My Drive/Gen_AI/OPENAI_API_Key.txt", "r") as f:
    api_key = f.read().strip()

# Set the API key for OpenAI
openai.api_key = api_key


Mounted at /content/drive


In [10]:
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_ai_response(criterion,subcriterion):
    """
    Generate a response using gpt-4o-mini ChatCompletion based on the user query and retrieved information.
    """
    system_content="You are a helpful AI assistant in the Financial Technology consulting domain, specialized in providing accurate answers."

    user_content= f"""
                  Imagine a panel discussion featuring four industry experts, each bringing specialized insights into the migration of a US-based financial institution’s legacy system to Salesforce Experience Cloud. The institution currently operates on a custom-built system using Java, Angular, FTL, Oracle, and custom APIs. The objective is to modernize operations, enhance customer and partner engagement, ensure compliance, and maximize Return on Investment (ROI).

                  Expert Panelists:
                  Salesforce Solution Architect – Specializes in migrating legacy systems to Salesforce while ensuring data integrity, security, and scalability.
                  Financial Industry Consultant – Provides expertise in regulatory compliance (GDPR, CCPA, GLBA, SOX), risk mitigation, and security requirements for US financial institutions.
                  Legacy System Migration Specialist – Focuses on transitioning from Java, Angular, Oracle-based systems to Salesforce Experience Cloud, addressing data migration, technical debt, and operational continuity.
                  Customer Experience (CX) Strategist – Ensures the new solution delivers a seamless, user-friendly experience, improves engagement, and drives business value.
                  Discussion Focus:
                  Main Topic: {criterion}
                  Sub-Topic: {subcriterion}

                  The discussion evaluates the adoption of Salesforce Experience Cloud as an alternative to the current home-grown solution while addressing key challenges such as minimizing disruption, ensuring compliance, enhancing digital experiences, and optimizing ROI.

                  Each expert provides a unique perspective:

                  Solution Architect – Technical challenges, API integrations, scalability, and security.
                  Industry Consultant – Compliance, security, and risk management.
                  Migration Specialist – Transition complexities, downtime mitigation, and resource upskilling.
                  CX Strategist – User experience, personalization, and engagement strategies.
                  Deliverables:
                  The panel will collaboratively define evaluation criteria, subcriteria, and parameters, assigning an importance scale (1-5) to assess their impact.

                  Structured Output:
                  A Markdown-style table with four columns, strictly in this order:
                  Evaluation Criterion – The main topic ({criterion}).
                  Importance Scale – A score (1-5) indicating {criterion}’s significance.
                  Evaluation Subcriterion – The sub-topic ({subcriterion}).
                  Parameters of the Subcriterion – Specific parameters used to evaluate {subcriterion}.

                  Each subcriterion must have top, at least 7 parameters and maximum 10 parameters.These parameter must be comma(,) saperate values for evaluation.
                  In Table's row  must not repeat Criterion and subsriterion.
                  The discussion will cover key challenges, proposed solutions, and success measurement metrics to ensure an informed transition to Salesforce Experience Cloud.
                  """

    messages = [
        {"role": "system", "content": system_content },
        {"role": "user", "content": user_content },
    ]

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    # Accessing content from the response object using .choices attribute
    content = response.choices[0].message.content
    print(content)

    return content


In [11]:
import pandas as pd
from datetime import datetime
import re

# Step 1: Extract Table Data from Markdown
def extract_table(content):
    lines = content.split("\n")
    table_lines = [line for line in lines if "|" in line]  # Extract only table lines
    if len(table_lines) < 3:
        return None  # No valid table found

    headers = [h.strip() for h in table_lines[0].split("|")[1:-1]]  # Extract headers
    data = []
    for line in table_lines[2:]:  # Skip the header and separator line
        values = [v.strip() for v in line.split("|")[1:-1]]
        if values:
            data.append(values)

    return headers, data

# Step 2: Extract Discussion Summary and ROI Framework
def extract_section(content, section_title):
    match = re.search(f"### {section_title}(.*?)(###|$)", content, re.DOTALL)
    return match.group(1).strip() if match else ""


# def save_to_excel(headers, data, discussion, roi, filename):
def save_to_excel(headers, data, filename):

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename}_{timestamp}.xlsx"
    filepath = f"/content/drive/My Drive/Gen_AI/{filename}"

    # Concatenate all DataFrames in the list
    all_data_df = pd.concat(data, ignore_index=True)

    with pd.ExcelWriter(filepath, engine="openpyxl") as writer:
        # Save Table Data
        if headers and not all_data_df.empty:  # Check if 'all_data_df' DataFrame is not empty
            all_data_df.to_excel(writer, sheet_name="Evaluation Criteria", index=False) # Write DataFrame directly


    print(f"✅ Report saved successfully as: {filepath}")



In [12]:
# To generate the excels

import pandas as pd
from numpy import NaN

df_all = []

for criterion, subcriteria in criteria_list.items():
    df_criterion=[]
    for item in subcriteria:
        content = generate_ai_response(criterion, item)
        headers, data = extract_table(content)
        # Convert 'data' to a DataFrame to use iterrows
        df_data = pd.DataFrame(data, columns=headers)  # Create a DataFrame from data and headers
        df_all.append(df_data)
        df_criterion.append(df_data)
        print("df_data --->\n",df_data)
    # Generate a unique sheet name based on the criterion and loop index
    sheet_name = f"{criterion}-Evaluation Criteria"
    save_to_excel(headers, df_criterion, sheet_name) # Pass the sheet name to the function

sheet_name = f"Evaluation Criteria - {criterion}"
save_to_excel(headers, df_all, "All_criterion") # Pass the sheet name to the function



In [13]:
def expand_rows(input_file, output_file):
    # Load the Excel file
    df = pd.read_excel(input_file)

    # Ensure the necessary column exists
    column_to_split = "Parameters of the Subcriterion"
    if column_to_split not in df.columns:
        raise ValueError(f"Column '{column_to_split}' not found in the input file.")

    # Expand rows
    df_expanded = df.assign(**{column_to_split: df[column_to_split].str.split(',')})
    df_expanded = df_expanded.explode(column_to_split)

    # Trim spaces from the split values
    df_expanded[column_to_split] = df_expanded[column_to_split].str.strip()

    # Save to new Excel file
    df_expanded.to_excel(output_file, index=False)
    print(f"Expanded file saved as: {output_file}")


In [14]:
import pandas as pd
import re
import os

def get_matching_files(folder_path, search_value):
    matching_files = []
    # List all files in the folder
    for file_name in os.listdir(folder_path):
        if search_value in file_name:
            matching_files.append(file_name)
    return matching_files

def expand_rows(input_file, output_file, sheet_name):
    # Load the Excel file
    df = pd.read_excel(input_file)
    # Ensure the necessary column exists
    column_to_split = "Parameters of the Subcriterion"
    if column_to_split not in df.columns:
        raise ValueError(f"Column '{column_to_split}' not found in the input file.")
    # Expand rows
    df_expanded = df.assign(**{column_to_split: df[column_to_split].str.split(',')})
    df_expanded = df_expanded.explode(column_to_split)
    # Trim spaces from the split values
    df_expanded[column_to_split] = df_expanded[column_to_split].str.strip()
    # Save to new Excel file with specified sheet name

    # **Instead of appending, write all dataframes to a dictionary and save at once**
    if not os.path.exists(output_file):
        all_dfs = {sheet_name: df_expanded}  # Initialize with the first sheet
    else:
        all_dfs = pd.read_excel(output_file, sheet_name=None)  # Read all sheets
        all_dfs[sheet_name] = df_expanded  # Add the current sheet

    # **Write all sheets to the output file**
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for sheet_name, df in all_dfs.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"Expanded file saved as: {output_file}, sheet name: {sheet_name}")

def extract_criterion(filename):
    # Modified regex to handle "All_criterion"
    match = re.match(r"^(.*?)(?:-Evaluation Criteria|_).*", filename)
    if match:
      return match.group(1)
    else:
      return None

In [15]:
folder_path = "/content/drive/My Drive/Gen_AI/"  # Replace with your folder path
search_value = "20250222"
matching_files = get_matching_files(folder_path, search_value)

for file_name in matching_files:
  input_file = f"/content/drive/My Drive/Gen_AI/{file_name}"  # Replace with your actual file name
  sheet_name = extract_criterion(file_name)
  output_file = f"/content/drive/My Drive/Gen_AI/Out/All_In_One.xlsx"  # Output file name
  print("input_file-->",input_file)
  print("output_file-->",output_file)
  print("sheet_name-->",sheet_name)
  expand_rows(input_file, output_file, sheet_name)